## predicting the values of air pollution (Regression)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt 
import pandas as pd  
import seaborn as sns 
from sklearn.datasets import load_boston
import warnings
warnings.filterwarnings('ignore')
plt.style.use('ggplot')

In [ ]:
train = pd.read_csv('../input/tabular-playground-series-jul-2021/train.csv')
test = pd.read_csv('../input/tabular-playground-series-jul-2021/test.csv')
print('train shape:',train.shape)
print('test shape:',test.shape)

In [ ]:
# found that one of the test data is in the train set and removed it
# Dropping the last row which is 2011-01-01 00:00:00
train=train.loc[~(train['date_time']=='2011-01-01 00:00:00')].reset_index(drop=True)

In [ ]:
print('train shape:',train.shape)

In [ ]:
all_data = pd.concat([train, test])
# convert to datatime format
all_data['date_time'] = pd.to_datetime(all_data['date_time'])
all_data.head()

## EDA
- The distribution of deg_C shows peaks between 20 to 30 deg.
- There is a dip in relative humidity at 40% and there are two peaks at 30% and 45% approx.
- The absolute humidity value shows peaks at 0.25g/m3(i have assumed it to be g/m3.Data info did not explicitly mention any units).
- The distribution of sensor_1,2,3 & 5 appears to be left skewed whereas sensor-4 is normal with outliers at 500.

In [ ]:
fig,ax=plt.subplots(4,2,figsize=(20,15))
for i,col in enumerate(train.columns[1:9]):
    ax[i%4][i//4].hist(train[col],bins=40,color='darkblue',label=f'{col}')
    ax[i%4][i//4].set_title(f'Distribution of {col}',fontsize=15)
    ax[i%4][i//4].set_xlabel(f'{col}')
    ax[i%4][i//4].set_ylabel('Dist')
    plt.subplots_adjust(hspace=0.45)

In [ ]:
fig,ax=plt.subplots(3,1,figsize=(8,10))
for i,col in enumerate(train.columns[9:12]):
    ax[i%3].hist(train[col],bins=40,color='darkblue',label=f'{col}')
    ax[i%3].set_title(f'Distribution of {col}',fontsize=15)
    ax[i%3].set_xlabel(f'{col}')
    ax[i%3].set_ylabel('Dist')
    plt.subplots_adjust(hspace=0.45)

### 分析每天的資訊
Looking at the day wise trend,we see that there has been sudden peak and dips for certain days over the month.While the temperatures have been above 20 deg after late may, there is a dip in temperature less than 15 dec after Nov but there is a sudden increase in mid december.

In [ ]:
# 字串轉換日期格式
train['date_time']=pd.to_datetime(train['date_time'],format='%Y-%m-%d %H:%M:%S')
test['date_time']=pd.to_datetime(test['date_time'],format='%Y-%m-%d %H:%M:%S')
# Following code is inspired from - https://www.kaggle.com/nroman/eda-for-ashrae
fig,ax=plt.subplots(1,1,figsize=(12,6))
train[['date_time','deg_C']].set_index('date_time').resample('D').mean()['deg_C'].plot(ax=ax,label='by hour(train)',alpha=1,color='blue').set_ylabel('deg C',fontsize=10)
ax.set_title('Trend of Mean deg_C by Day',fontsize=12)
ax.set_xlabel('')

For the test set, if we try to compare between March month of train, we could see the temperatures have started from approx 3 deg and increased above 15 deg.

In [ ]:
fig,ax=plt.subplots(1,1,figsize=(12,6))
test[['date_time','deg_C']].set_index('date_time').resample('D').mean()['deg_C'].plot(ax=ax,label='by hour(train)',alpha=1,color='blue').set_ylabel('deg C',fontsize=10)
ax.set_title('Trend of Mean deg_C by Day',fontsize=12)
ax.set_xlabel('')

## 資料前處理
進一步處理之前先確認是否有缺失值：

In [ ]:
all_data.isnull().sum()

In [ ]:
# all_data['hr'] = all_data.date_time.dt.hour*60+all_data.date_time.dt.minute
# all_data['day'] =all_data.date_time.dt.weekday//5
# all_data['satday'] = all_data.date_time.dt.weekday==5
# all_data['hr1'] = all_data.date_time.dt.hour*60+all_data.date_time.dt.minute

In [ ]:
# all_data['year'] = all_data['date_time'].dt.year
# all_data['month'] = all_data['date_time'].dt.month
# all_data['week'] = all_data['date_time'].dt.week
# all_data['day'] = all_data['date_time'].dt.day
# all_data['dayofweek'] = all_data['date_time'].dt.dayofweek
# all_data['hour'] = all_data['date_time'].dt.hour
# # convert datetime to timestamp(s)
# all_data['time'] = all_data['date_time'].astype(np.int64)//10**9
# all_data.drop(columns = 'date_time', inplace = True)
# print('all_data shape:', all_data.shape)
# all_data.head()

In [ ]:
## New idea
all_data["hour"] = all_data["date_time"].dt.hour
all_data["working_hours"] =  all_data["hour"].isin(np.arange(8, 21, 1)).astype("int")
all_data["is_weekend"] = (all_data["date_time"].dt.dayofweek >= 5).astype("int")
all_data.drop(columns = 'hour', inplace = True)
# convert datetime to timestamp(s)
all_data['time'] = all_data['date_time'].astype(np.int64)//10**9
all_data.drop(columns = 'date_time', inplace = True)

In [ ]:
X=all_data[:len(train)].drop(columns = ['target_carbon_monoxide', 'target_benzene', 'target_nitrogen_oxides'])
y=all_data[:len(train)][['target_carbon_monoxide', 'target_benzene', 'target_nitrogen_oxides']]
y_log=np.log1p(y)
X_test=all_data[len(train):].drop(columns = ['target_carbon_monoxide', 'target_benzene', 'target_nitrogen_oxides'])
print('X_train shape:', X.shape)
print('y_train shape:', y.shape)
print('X_test shape:', X_test.shape)

### Standardization 平均&變異數標準化
將所有特徵標準化，也就是高斯分佈。使得數據的平均值為0，方差為1。
適合的使用時機於當有些特徵的方差過大時，使用標準化能夠有效地讓模型快速收斂。

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler().fit(X)
X_scaled = scaler.transform(X)
X_test_scaled = scaler.transform(X_test)

In [ ]:
fig,ax=plt.subplots(4,2,figsize=(20,15))
for i in range(8):
    col=train.columns[1:9][i]
    ax[i%4][i//4].hist(X_test_scaled[:,i],bins=40,color='darkblue',label=f'{col}')
    ax[i%4][i//4].set_title(f'Distribution of {col}',fontsize=15)
    ax[i%4][i//4].set_xlabel(f'{col}')
    ax[i%4][i//4].set_ylabel('Dist')
    plt.subplots_adjust(hspace=0.45)

In [ ]:
fig,ax=plt.subplots(3,1,figsize=(8,10))
for i in range(3):
    col=train.columns[9:12][i]
    ax[i%3].hist(y_log[:][col],bins=40,color='darkblue',label=f'{col}')
    ax[i%3].set_title(f'Distribution of {col}',fontsize=15)
    ax[i%3].set_xlabel(f'{col}')
    ax[i%3].set_ylabel('Dist')
    plt.subplots_adjust(hspace=0.45)

## XGBoost(回歸器)
Boosting 則是希望能夠由後面生成的樹，來修正前面樹學的不好的地方。

Parameters:
- n_estimators: 總共迭代的次數，即決策樹的個數。預設值為100。
- max_depth: 樹的最大深度，默認值為6。
- booster: gbtree 樹模型(預設) / gbliner 線性模型
- learning_rate: 學習速率，預設0.3。
- gamma: 懲罰項係數，指定節點分裂所需的最小損失函數下降值。

Attributes:
- feature_importances_: 查詢模型特徵的重要程度。

Methods:
- fit: 放入X、y進行模型擬合。
- predict: 預測並回傳預測類別。
- score: 預測成功的比例。
- predict_proba: 預測每個類別的機率值。

In [ ]:
import xgboost as xgb
from sklearn.multioutput import MultiOutputRegressor

# 建立xgbrModel模型
xgbrModel=xgb.XGBRegressor(n_estimators=100)
# 使用訓練資料訓練模型
model = MultiOutputRegressor(xgbrModel).fit(X_scaled,y_log)

### 模型評估
scikit-learn 決策樹迴歸模型的score函式是R2 score，可作為模型評估依據，其數值越接近於1代表模型越佳。
除了R2 score還有其他許多回歸模型的評估方法，例如： MSE、MAE、RMSE。

In [ ]:
from sklearn import metrics
# 使用訓練資料預測
y_pred=model.predict(X_scaled)
print("訓練集")
print('R2 score: ', model.score(X_scaled,y_log))
mse = metrics.mean_squared_error(y_log, y_pred)
print('MSE score: ', mse)

## 預測輸出
輸出 Y log1p()對數去偏 - 使用自然對數去除偏態，先加1再取對數，還原時先取指數後再減1。對於可能出現等於零的資料使用。

In [ ]:
sub = pd.read_csv('../input/tabular-playground-series-jul-2021/sample_submission.csv')
y_pred=model.predict(X_test_scaled)
y_pred=np.expm1(y_pred)
sub['target_carbon_monoxide'] = y_pred[:,0]
sub['target_benzene'] = y_pred[:,1]
sub['target_nitrogen_oxides'] = y_pred[:,2]
sub.head()

In [ ]:
sub.to_csv('submission.csv', index=False)